## Quantitative Risk Management Assignement
Giuliano Cunti <br>
Lukas Schreiner

In [1]:
#Import Libraries
import numpy as np
import pandas as pd
from copulas.bivariate.base import Bivariate

In [2]:
#Import Data MXEUC Index
data1 = pd.read_excel('qrm18HSG_assignmentdata.xlsx', usecols = "A,B" ,names = [ 'Date', 'MXEUC Index']).set_index('Date')
data1 = data1.iloc[1:]
#Import Data SPX Index
data2 = pd.read_excel('qrm18HSG_assignmentdata.xlsx', usecols = "C,D" ,names = [ 'Date', 'SPX Index']).set_index('Date')
data2 = data2.iloc[1:]

#Merge Data
data = pd.concat([data1, data2], axis = 1, join_axes = [data1.index])

#Convert to numerical values interpolate missing values
data['SPX Index'] = pd.to_numeric(data['SPX Index'])
data['MXEUC Index'] = pd.to_numeric(data['MXEUC Index'])
data['SPX Index'] = data['SPX Index'].interpolate()

#Calculate Returns of the Indices
data['MXEUC Returns'] = data['MXEUC Index']/data['MXEUC Index'].shift(1) - 1
data['SPX Index Returns'] = data['SPX Index']/data['SPX Index'].shift(1) - 1

#Only take the newest 500 Oberservations
data = data.iloc[1:501]

data.head()

,MXEUC Index,SPX Index,MXEUC Returns,SPX Index Returns
Date,,,,
2005-01-03,89.6602,1202.0891,0.005159,-0.008109
2005-01-04,90.0505,1188.1801,0.004353,-0.011571
2005-01-05,89.4664,1183.9815,-0.006486,-0.003534
2005-01-06,89.9265,1188.3980,0.005143,0.003730
2005-01-07,90.6058,1186.7126,0.007554,-0.001418


In [3]:
#Calculate Sample Means and Variances
means = [np.mean(data['MXEUC Returns']), np.mean(data['SPX Index Returns'])]
print(means)
variances = [np.var(data['MXEUC Returns']), np.var(data['SPX Index Returns'])]
print(variances)

[0.0007688865556201618, 0.00037513875090646164]
[5.061087267757628e-05, 3.915794768807607e-05]


In [4]:
#Calculate Covariance Matrix
cov_mat = np.cov(data['MXEUC Returns'], data['SPX Index Returns'], ddof = 0)
print(cov_mat)

[[5.06108727e-05 1.92605961e-05]
 [1.92605961e-05 3.91579477e-05]]


In [5]:
#Create an array that contains only the return data
return_data = np.array([data['MXEUC Returns'], data['SPX Index Returns']]).T

In [6]:
#Fit a bivariate gumbel copula
#The copulas lirary from Data to AI Lab at MIT is used
#https://pypi.org/project/copulas/

gumbel = Bivariate(copula_type='Gumbel')
gumbel.fit(return_data)
gumbel.compute_theta()

1.3831731143911365